###  In this notebook , model testing is automated in for-loops,
### This notebook is created for optimization of model testing procedure

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from sklearn import linear_model

In [4]:
## Data loading procedure

In [2]:
X_2house = pd.read_csv('/wgdisk/ho0338/ek79/Tien/X_2house_v0.csv')
X_2house=X_2house.drop(columns=['Unnamed: 0'])
X_2house.index.rename('Order', inplace=True)

In [181]:
y_2house = -1*np.load('Data/Processed/y_2house.npy')

In [4]:
## check on the data dimensions
print('X_input:',X_2house.shape)
print('y_input:',y_2house.shape)

X_input: (4290985, 158)
y_input: (4290985,)


In [355]:
## Select only 2 features from X
tt_att = 'Kitchen Qual'
X_input = X_2house[[tt_att,tt_att+'_2']]

In [356]:
X_input.describe()

,Kitchen Qual,Kitchen Qual_2
count,4.290985e+06,4.290985e+06
mean,3.531044e+00,3.491482e+00
std,6.698706e-01,6.555604e-01
min,1.000000e+00,1.000000e+00
25%,3.000000e+00,3.000000e+00
50%,3.000000e+00,3.000000e+00
75%,4.000000e+00,4.000000e+00
max,5.000000e+00,5.000000e+00


In [357]:
## setup train-test split
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_input, y_2house, test_size=0.5, random_state=12)

In [358]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2145492, 2)
(2145492,)
(2145493, 2)
(2145493,)


In [359]:
### setup model parameters
#mod_type = ['Lasso','Ridge']
mod_type = ['Lasso','Ridge']
fit_int, norm, copyX, max_itr, tolerance, solv, Precomp, pos, r_stat = False, False, True, None, 0.001, 'auto', False, False, 12
test_alph = [5.0,2.0,1.0,0.1]

In [360]:
# prepare the numpy array to store the model coefficients & intercepts
k = X_train.shape[1]
r_k = len(mod_type)*len(test_alph)
col_names =['model_type','alpha','coef_'+tt_att,'coef_'+tt_att+'_2','intercept','MSE_train','MSE_test','MSE_reduction_ratio']

In [361]:
#  initiate a new dataframe , empty to append
df= pd.DataFrame(columns = col_names)
df.head()

,model_type,alpha,coef_Kitchen Qual,coef_Kitchen Qual_2,intercept,MSE_train,MSE_test,MSE_reduction_ratio


In [309]:
pd.set_option('display.max_columns', None)

In [362]:
from sklearn import metrics
for test_type in tqdm(mod_type):
    print('model type: ',test_type)
    for alph in tqdm(test_alph):
        print('alpha is: ', alph)
        if test_type == 'Lasso': 
            model = linear_model.Lasso(alpha=alph, fit_intercept=fit_int, normalize=norm, precompute=Precomp, copy_X=copyX, max_iter=1000, random_state=r_stat, selection='cyclic')
        if test_type == 'Ridge': 
            model = linear_model.Ridge(alpha=alph, fit_intercept=fit_int, normalize=norm, copy_X= copyX, max_iter= max_itr, tol= tolerance, solver= solv, random_state=r_stat)
        model.fit(X_train,y_train)
        y_test_pred = model.predict(X_test)
        y_train_pred = model.predict(X_train)
        MSE_test= metrics.mean_squared_error(y_test, y_test_pred)
        MAE_test= metrics.mean_absolute_error(y_test, y_test_pred)
        MSE_train= metrics.mean_squared_error(y_train, y_train_pred)
        MAE_train= metrics.mean_absolute_error(y_train, y_train_pred)
        MSE_neg_ratio= (MSE_test-MSE_train)/MSE_train*100
        print('MSE score(testing):', MSE_test)
        print('MAE score(testing):', MAE_test)
        print('MSE score(train):', MSE_train)
        print('MAE score(train):', MAE_train)
        print('MSE reduction ratio:', MSE_neg_ratio)
        # save coefficients
        tmp_row=[test_type,alph]+list(model.coef_)+[model.intercept_]+[MSE_train, MSE_test, MSE_neg_ratio]
        df.loc[len(df)]=tmp_row


  0%|          | 0/4 [00:00<?, ?it/s]

model type:  Lasso
alpha is:  5.0



 25%|██▌       | 1/4 [00:01<00:04,  1.64s/it]

MSE score(testing): 6995341398.89
MAE score(testing): 61880.057473
MSE score(train): 6973003862.74
MAE score(train): 61765.2244348
MSE reduction ratio: 0.320343091475
alpha is:  2.0



 50%|█████     | 2/4 [00:03<00:03,  1.70s/it]

...

MSE score(testing): 6995339760.6
MAE score(testing): 61880.6282656
MSE score(train): 6973003751.48
MAE score(train): 61765.7957901
MSE reduction ratio: 0.320321197495
alpha is:  0.1



 50%|█████     | 2/4 [00:00<00:00, 10.57it/s]

MSE score(testing): 6995339412.42
MAE score(testing): 61880.7567187
MSE score(train): 6973003746.9
MAE score(train): 61765.9243595
MSE reduction ratio: 0.320316270296
model type:  Ridge
alpha is:  5.0
MSE score(testing): 6995339445.75
MAE score(testing): 61880.744292
MSE score(train): 6973003747.01
MAE score(train): 61765.9119219
MSE reduction ratio: 0.320316746507
alpha is:  2.0
MSE score(testing): 6995339402.77
MAE score(testing): 61880.7603171
MSE score(train): 6973003746.87
MAE score(train): 61765.9279611
MSE reduction ratio: 0.320316132066



100%|██████████| 2/2 [00:07<00:00,  3.90s/it]

alpha is:  1.0
MSE score(testing): 6995339388.46
MAE score(testing): 61880.7656588
MSE score(train): 6973003746.86
MAE score(train): 61765.9333076
MSE reduction ratio: 0.32031592725
alpha is:  0.1
MSE score(testing): 6995339375.6
MAE score(testing): 61880.7704664
MSE score(train): 6973003746.85
MAE score(train): 61765.9381194
MSE reduction ratio: 0.320315742914


class sklearn.linear_model.Lasso(alpha=1.0, fit_intercept=True, normalize=False, precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection=’cyclic’)

class sklearn.linear_model.Ridge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver=’auto’, random_state=None)

In [363]:
df

,model_type,alpha,coef_Kitchen Qual,coef_Kitchen Qual_2,intercept,MSE_train,MSE_test,MSE_reduction_ratio
0,Lasso,5.0,-80995.327218,81015.287979,0.0,6.973004e+09,6.995341e+09,0.320343
1,Lasso,2.0,-81002.178332,81022.218860,0.0,6.973004e+09,6.995340e+09,0.320327
2,Lasso,1.0,-81004.462396,81024.529505,0.0,6.973004e+09,6.995340e+09,0.320321
3,Lasso,0.1,-81006.517654,81026.608694,0.0,6.973004e+09,6.995339e+09,0.320316
4,Ridge,5.0,-81006.318914,81026.407498,0.0,6.973004e+09,6.995339e+09,0.320317
5,Ridge,2.0,-81006.575263,81026.666914,0.0,6.973004e+09,6.995339e+09,0.320316
6,Ridge,1.0,-81006.660713,81026.753387,0.0,6.973004e+09,6.995339e+09,0.320316
7,Ridge,0.1,-81006.737618,81026.831212,0.0,6.973004e+09,6.995339e+09,0.320316


In [369]:
## Experiment on the effect of coefficients
case1=pd.DataFrame(columns = X_input.columns)
case1.loc[0]=[2,5]
case1.head()

,Kitchen Qual,Kitchen Qual_2
0,2,5


In [370]:
pred_pric = model.predict(case1)

In [371]:
print(pred_pric)

[ 243120.6808237]


In [372]:
pred_pric_m = model.intercept_+ (model.coef_[0]*case1.iloc[0,0])+(model.coef_[1]*case1.iloc[0,1])

In [373]:
print(pred_pric_m)

243120.680824


In [378]:
###Verify full model coefficients
LR_coef = pd.read_csv('Experiments_result/LR_models_hyperparam_saveout.csv')
LR_coef=LR_coef.drop(columns=['Unnamed: 0'])

In [407]:
LR_coef_f = LR_coef.loc[0]*

In [408]:
LR_coef.head()

,model_type,alpha,coef_MS SubClass,coef_MS Zoning,coef_Lot Frontage,coef_Lot Area,coef_Street,coef_Alley,coef_Lot Shape,coef_Land Contour,coef_Utilities,coef_Lot Config,coef_Land Slope,coef_Neighborhood,coef_Condition 1,coef_Condition 2,coef_Bldg Type,coef_House Style,coef_Overall Qual,coef_Overall Cond,coef_Year Built,coef_Year Remod/Add,coef_Roof Style,coef_Roof Matl,coef_Exterior 1st,coef_Exterior 2nd,coef_Mas Vnr Type,coef_Mas Vnr Area,coef_Exter Qual,coef_Exter Cond,coef_Foundation,coef_Bsmt Qual,coef_Bsmt Cond,coef_Bsmt Exposure,coef_BsmtFin Type 1,coef_BsmtFin SF 1,coef_BsmtFin Type 2,coef_BsmtFin SF 2,coef_Bsmt Unf SF,coef_Total Bsmt SF,coef_Heating,coef_Heating QC,coef_Central Air,coef_Electrical,coef_1st Flr SF,coef_2nd Flr SF,coef_Low Qual Fin SF,coef_Gr Liv Area,coef_Bsmt Full Bath,coef_Bsmt Half Bath,coef_Full Bath,coef_Half Bath,coef_Bedroom AbvGr,coef_Kitchen AbvGr,coef_Kitchen Qual,coef_TotRms AbvGrd,coef_Functional,coef_Fireplaces,coef_Fireplace Qu,coef_Garage Type,coef_Garage Yr Blt,coef_Garage Finish,coef_Garage Cars,coef_Garage Area,coef_Garage Qual,coef_Garage Cond,coef_Paved Drive,coef_Wood Deck SF,coef_Open Porch SF,coef_Enclosed Porch,coef_3Ssn Porch,coef_Screen Porch,coef_Pool Area,coef_Pool QC,coef_Fence,coef_Misc Feature,coef_Misc Val,coef_Mo Sold,coef_Yr Sold,coef_Sale Type,coef_Sale Condition,coef_MS SubClass_2,coef_MS Zoning_2,coef_Lot Frontage_2,coef_Lot Area_2,coef_Street_2,coef_Alley_2,coef_Lot Shape_2,coef_Land Contour_2,coef_Utilities_2,coef_Lot Config_2,coef_Land Slope_2,coef_Neighborhood_2,coef_Condition 1_2,coef_Condition 2_2,coef_Bldg Type_2,coef_House Style_2,coef_Overall Qual_2,coef_Overall Cond_2,coef_Year Built_2,coef_Year Remod/Add_2,coef_Roof Style_2,coef_Roof Matl_2,coef_Exterior 1st_2,coef_Exterior 2nd_2,coef_Mas Vnr Type_2,coef_Mas Vnr Area_2,coef_Exter Qual_2,coef_Exter Cond_2,coef_Foundation_2,coef_Bsmt Qual_2,coef_Bsmt Cond_2,coef_Bsmt Exposure_2,coef_BsmtFin Type 1_2,coef_BsmtFin SF 1_2,coef_BsmtFin Type 2_2,coef_BsmtFin SF 2_2,coef_Bsmt Unf SF_2,coef_Total Bsmt SF_2,coef_Heating_2,coef_Heating QC_2,coef_Central Air_2,coef_Electrical_2,coef_1st Flr SF_2,coef_2nd Flr SF_2,coef_Low Qual Fin SF_2,coef_Gr Liv Area_2,coef_Bsmt Full Bath_2,coef_Bsmt Half Bath_2,coef_Full Bath_2,coef_Half Bath_2,coef_Bedroom AbvGr_2,coef_Kitchen AbvGr_2,coef_Kitchen Qual_2,coef_TotRms AbvGrd_2,coef_Functional_2,coef_Fireplaces_2,coef_Fireplace Qu_2,coef_Garage Type_2,coef_Garage Yr Blt_2,coef_Garage Finish_2,coef_Garage Cars_2,coef_Garage Area_2,coef_Garage Qual_2,coef_Garage Cond_2,coef_Paved Drive_2,coef_Wood Deck SF_2,coef_Open Porch SF_2,coef_Enclosed Porch_2,coef_3Ssn Porch_2,coef_Screen Porch_2,coef_Pool Area_2,coef_Pool QC_2,coef_Fence_2,coef_Misc Feature_2,coef_Misc Val_2,coef_Mo Sold_2,coef_Yr Sold_2,coef_Sale Type_2,coef_Sale Condition_2,intercept,MSE_train,MSE_test,MSE_reduction_ratio
0,Lasso,5.0,-129.096329,-3381.508762,33.897681,0.378656,12006.857528,1773.737877,-427.939799,1435.067699,2419.185652,248.571051,-4499.015420,161.171232,-18.985918,3167.925479,-620.618564,-9824.430515,11565.960015,4161.712532,-37.635419,-6.422364,1654.237535,-193.337284,-459.228837,15.479760,4735.840039,43.303614,12656.960710,-1057.105255,4225.384087,9706.655054,-10779.045244,2370.358729,513.666464,29.977300,-74.637165,23.166834,14.979953,-2.175342,-2523.033815,2389.133569,-651.138267,-1208.386859,48.727911,59.085258,31.115817,-8.184609,3326.348761,-3506.024700,5161.737942,3722.501662,-5167.529966,-15591.490263,8980.644870,2872.014362,2049.255462,2545.047516,373.903891,936.849904,-98.764782,2183.022033,9739.363433,11.871922,-0.277890,-2633.112666,1426.190660,13.605374,-9.174842,-1.557694,-19.517561,44.566957,-250.554522,21594.047695,-288.426621,4138.765362,-5.344538,-111.437712,-7.372428,725.096981,-2022.575834,157.666615,1699.348512,-6.084111,-0.261714,-4740.083131,-3292.251646,1322.071808,-1436.230435,-1600.961662,-488.588394,7538.234085,-319.477633,-1095.788040,3876.287975,-258.523584,14519.922763,-12454.5

In [409]:
LR_coef_f[160]

-0.0

In [423]:
# form a function to calculate prediciton:
def pred_cal(X_in,coef,y_true):
    y_pred =0
    for i in range(len(X_in)):
        y_pred = y_pred + X_in[i]*coef[i]
    y_pred = y_pred+coef[len(coef)-1]
    print("prediction price difference:", y_pred)
    print("true price difference: ", y_true)
    return y_pred

In [432]:
X_in = list(X_2house.iloc[135])
y_true = y_2house[135]

In [433]:
tmp = zip(X_2house.columns, X_in, range(158))
for ind,item,id in tmp:
    print(ind,item, id)

MS SubClass 20.0 0
MS Zoning 4.0 1
Lot Frontage 141.0 2
Lot Area 31770.0 3
Street 1.0 4
Alley 1.0 5
Lot Shape 0.0 6
Land Contour 3.0 7
Utilities 4.0 8
Lot Config 0.0 9
Land Slope 2.0 10
Neighborhood 15.0 11
Condition 1 2.0 12
Condition 2 2.0 13
Bldg Type 0.0 14
House Style 1.0 15
Overall Qual 6.0 16
Overall Cond 5.0 17
Year Built 1960.0 18
Year Remod/Add 1960.0 19
Roof Style 3.0 20
Roof Matl 1.0 21
Exterior 1st 3.0 22
Exterior 2nd 10.0 23
Mas Vnr Type 4.0 24
Mas Vnr Area 112.0 25
Exter Qual 3.0 26
Exter Cond 3.0 27
Foundation 1.0 28
Bsmt Qual 3.0 29
Bsmt Cond 4.0 30
Bsmt Exposure 4.0 31
BsmtFin Type 1 4.0 32
BsmtFin SF 1 639.0 33
BsmtFin Type 2 1.0 34
BsmtFin SF 2 0.0 35
Bsmt Unf SF 441.0 36
Total Bsmt SF 1080.0 37
Heating 1.0 38
Heating QC 2.0 39
Central Air 1.0 40
Electrical 5.0 41
1st Flr SF 1656.0 42
2nd Flr SF 0.0 43
Low Qual Fin SF 0.0 44
Gr Liv Area 1656.0 45
Bsmt Full Bath 1.0 46
Bsmt Half Bath 0.0 47
Full Bath 1.0 48
Half Bath 0.0 49
Bedroom AbvGr 3.0 50
Kitchen AbvGr 1.0 51
K

In [434]:
# change specific attribute " Kitchen AbvGr"
X_in[129]=2

In [435]:
pred_cal(X_in,LR_coef_f[2:160],y_true)

prediction price difference: 47190.5485204
true price difference:  -35000.0


47190.548520423821